# Report - Finding similar items 

In [108]:
s1 = "abcdab"
s2 = "Får får lammdab"

# K-shingles

> A class&nbsp;<em>Shingling</em>&nbsp;that constructs&nbsp;<em>k</em>–shingles of a given length&nbsp;<em>k</em> (e.g., 10) from a given document, computes a hash value for each unique shingle and represents the document in the form of an ordered set of its hashed&nbsp;<em>k</em>-shingles.

* Between two documents
* Hash shingles
    * vad är en bra hash funktion? Finns i boken.

In [109]:
def shingle(s, k=5):
    """
    Split a string into k equal sizes 
    """
    return set([s[i:(i+k)].lower() for i in range(0, len(s)-k+1)])

Using Pyhon's native hash function with congruens to limit the possible outcomes.
This function returns a set of all unique functions. I think it is correct, according to [Stanford's slides](http://www.mmds.org/mmds/v2.1/ch03-lsh.pdf), page 22:

> Each **unique shingle** is a dimension

In [110]:
def get_hashed_shingles(document, k=5, hash_boundary = 2**32):
    shingles = shingle(document, k)
    hashed_shingles = [hash(shingle) % hash_boundary for shingle in shingles]
    return set(hashed_shingles)

In [111]:
# SANDBOX - setting test variables and testing functions
k = 3

print(shingle(s1, k=k))
h1 = get_hashed_shingles(s1, k=k)
print(h1)

print(shingle(s2, k=k))
h2 = get_hashed_shingles(s2, k=k)
print(h2)

{'abc', 'dab', 'bcd', 'cda'}
{3353848201, 1946142227, 1211333773, 2016924367}
{'får', 'r l', ' la', ' få', 'mda', 'amm', 'lam', 'dab', 'r f', 'mmd', 'år '}
{2272979106, 1437373442, 2270273381, 2178594277, 662651622, 2804253705, 1462665517, 1946142227, 2205822358, 3076531161, 3858275646}


# CompareSets

> A class&nbsp;<em>CompareSets </em>computes the Jaccard similarity of two sets of integers – two sets of hashed shingles.

From the lecture:

> Column similarirty is the Jaccard similarity of the corresponding sets (rows with value 1) and can be computed using bitwise AND (for intersection) and bitwise OR (for union)

Example:

If $C_1$ and $C_2$ are a transposed columns, we can follow this approach using bitwise OR and AND to find their Jaccardi Distance:

$$ 
\begin{align*}
C_1 & = 011010, \\ C_2 & = 101011 \\ 
C_1 \cap C_2 & = C_1  \land C_2 = 2 \\ 
C_1 \cup C_2 & = C_1 \lor C_2 = 5 \\ \\
\text{Jaccardi distance} & = \frac{|C_1 \cap C_2|}{|C_1 \cup C_2|} = \frac{2}{5}
\end{align*}
$$

When working with sets in Python, there is native support for bitwise comparisons with `&` and `|`.

In [112]:
# sandbox
import pandas as pd
import numpy as np

intersection = h1 & h2
union = h1 | h2

print(intersection)
print(union)
len(intersection) / len(union)


{1946142227}
{2272979106, 1437373442, 2270273381, 2178594277, 662651622, 3353848201, 2804253705, 1211333773, 1462665517, 2016924367, 1946142227, 2205822358, 3076531161, 3858275646}


0.07142857142857142

In [113]:
def similarity(A, B):
    """
    Jaccardi similarity between two sets of hashed shingles.
    """
    return len(A & B) / len(A | B)

def distance(A, B):
    """
    Jaccardi distance.
    """
    return 1-similarity(A, B)

In [114]:
# SANDBOX - setting test data
d3 = "Jag heter Linus Östlund och är en kille"
d4 = "Heter jag Linus Östlund och varför är jag en kille?"
d5 = "En kille heter Linus Östlund och är jag en kille?"

h3 = get_hashed_shingles(d3)
h4 = get_hashed_shingles(d4)
h5 = get_hashed_shingles(d5)

similarity(h3, h4)



0.3793103448275862

# MinHashing

> A class&nbsp;<em>MinHashing</em>&nbsp;that builds a minHash signature (in the form of a vector or a set) of a given length&nbsp;<em>n</em>&nbsp;from a given set of integers (a set of hashed shingles).


In [115]:
import numpy as np

def get_characteristic_matrix(documents_of_hashed_shingles):
    """
    Produce the characteristic matris.
    INPUT: documents_of_hashed_shingles -- a list of lists, where each element are a list of a document's hashed shingles.
    NOTE Total amount of shingles must be sorted!
    Returns a (shingles x documents) matrix with 0 and 1's
    TODO is it really necessary to compute this or should it be avoided? Might be very large and poor time complexity.
    TODO the input is not documents. Would it be better to just input the documents?
    """
    # Get the union of all shingles, which are the rows in the characteristic matrix
    union_of_shingles = get_union_of_shingles(documents_of_hashed_shingles)
    # Get the number of documents, which are the columns in the characteristic matrix
    number_of_documents = ['D'+str(i) for i in range(len(documents_of_hashed_shingles))]
    # Initiate an empty characteristic matrix and then populate it
    cm = np.zeros((len(union_of_shingles), len(documents_of_hashed_shingles)))
    for i, shingle in enumerate(union_of_shingles):
        for j, hashed_shingles in enumerate(documents_of_hashed_shingles):
            if shingle in hashed_shingles:
                cm[i][j] = 1
    # return a pandas dataframe for easier viewing
    return pd.DataFrame(cm, columns=number_of_documents, index=union_of_shingles, dtype=int) # kan sätta som 'bool' med dtype=bool

def get_union_of_shingles(shingles):
    """
    Get the union of all shingles from a list of shingles.
    Output is a sorted list of unique shingles.
    """
    union = set()
    for s in shingles:
        union |= s
    return sorted(union)

From a list of hashed shingles `[shingles of documents 1, shingles of document 2, ..]` we generate the *characteristic matrix*:

In [116]:
import pandas as pd

hashed_shingles = [h3, h4, h5]
df = get_characteristic_matrix(hashed_shingles)
df

,D0,D1,D2
35389113,0,1,0
177223151,0,1,0
220041776,1,0,0
322181731,1,1,1
385490113,1,1,1
...,...,...,...
4061221847,0,0,1
4075206348,1,1,1
4112375740,1,0,1
4167215095,1,0,0


# Generating hash functions
From [Stanford's slides, p. 39](http://www.mmds.org/mmds/v2.1/ch03-lsh.pdf):
> How to pick a random hash function $h(x)$?
> * $ h(x) = ( (ax + b) \mod p ) \mod N $, where $a$ and $b$ are random integers and $p$ is a prime number larger than $N$.

MinHash gives the signature of a set. The signature is then used as a proxy for the characterstic matrix, and allows a computational-friendly estimate for the Jaccardian distance. For big datasets, the law of large numbers kicks in.

For *k* (**k=100 recommended!**) independent hash functions, and according to the lecture, a hash function can look like this:

$$
h(x) \equiv (ax + b) \mod c
$$

> Pseudokod finnes i boken (och föreläsningen, 01:00:00 in!

You may also use permutations to achieve the same result, but the föreläsare recommends hash functions. Not sure which apporach is easier to understand. 

In [117]:
import random

# TODO fix magic number and approach all together

def get_hash_function(N):
    """
    Generate a hash function for a given N
    """
    a = random.randint(0, N)
    b = random.randint(0, N)
    p = get_prime(N)
    return lambda x: ((a * x + b) % p) % N

# TODO naive implementation, not efficient, but then again, it's a one off
def get_prime(N):
    """
    Get a prime number larger than N
    """
    n = random.randint(N*2, N*4)
    while not is_prime(n):
        n = random.randint(N*2, N*4)
    return n

def is_prime(n):
    """
    Check if n is a prime number
    """
    if n < 2:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

In [118]:
# SANDBOX
hash_functions = [get_hash_function(x) for x in range(10, 100, 7)]
len(hash_functions)

13

## Hashing the columns (documents) of the characteristic matrix

Here, I generate a `len(hash_functions)` x `len(documents)` matrix, where each row is a hash function, and each column is a document. The value in each cell $hash_i(index_{row})$

In [119]:
# from here on there it is mostly testing
data = []
for index in range(len(df.index)):
    data.append([f(index) for f in hash_functions])

print(len(data), len(data[0]))

df_hashed = pd.DataFrame(data=data, index=df.index, columns=range(len(hash_functions)))
#df_hashed
df_hashed

62 13


,0,1,2,3,4,5,6,7,8,9,10,11,12
35389113,8,11,23,14,32,44,6,53,46,13,41,86,13
177223151,8,9,18,25,17,41,47,51,56,48,56,37,5
220041776,5,7,2,5,2,27,36,36,0,10,71,75,91
322181731,5,13,21,16,25,24,25,34,10,45,6,26,72
385490113,2,11,16,27,10,10,14,19,20,7,21,64,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4061221847,3,6,20,14,33,14,50,27,12,0,63,61,45
4075206348,3,4,4,25,18,11,39,25,22,35,78,12,26
4112375740,0,2,23,5,3,8,37,10,32,70,13,50,18
4167215095,0,8,18,16,26,39,26,8,42,32,28,1,10


In [120]:
def compute_hashed_minhash(df, hash_functions):
    """
    df: a dataframe with rows as shingles and columns as documents.
    hash_functions: a list of hash functions.
    Compute the hashed minhash of a dataframe.
    """
    data = []
    for index in range(len(df.index)):
        data.append([f(index) for f in hash_functions])
    return pd.DataFrame(data=data, index=df.index, columns=range(len(hash_functions)))

In [123]:
def einsum_signature(A, B):
    """ 
    A method to get the minhash signature.
    Input:  A (shingles x document) matrix
            B (shingles x hash function values) matrix
    Output: (hash function values x document) matrix
    """
    # check that shingles match
    assert A.shape[0] == B.shape[0]
    # substitute all 0's with np.inf
    A = A.astype(float)
    A[A == 0] = np.inf
    # make a very large 3D array, using A to mask and filter values
    C = np.einsum('sd,sh->hsd', A, B)
    # np,nanmin reduces all candidate matrices to their lowest values
    # each einsum-generated matrix is a cell in the final array
    return np.nanmin(C, axis=1)

def naive_signature(A,B):
    """
    Keep for benchmarking
    """
    assert A.shape[0] == B.shape[0]
    signature = np.full((B.shape[1], A.shape[1]), np.inf)
    for i in range(A.shape[1]):
        for j in range(B.shape[1]):
            for k in range(A.shape[0]):
                if A[k][i] == 1:
                    signature[j][i] = min(signature[j][i], B[k][j])

In [124]:
A = df
B = compute_hashed_minhash(df, hash_functions)

# so A and B are dataframes....
print(einsum_signature(A.to_numpy(), B.to_numpy()))

print(naive_signature(A.to_numpy(), B.to_numpy()))




[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]
 [0. 1. 1.]
 [0. 0. 0.]
 [1. 0. 1.]
 [1. 0. 2.]
 [2. 0. 1.]]
None


In [23]:
# sandbox to test einsum
# exampl from the book
import numpy as np

# A = (shingels x documents)
A = np.array([[1,0,0,1], 
              [0,0,1,0], 
              [0,1,0,1], 
              [1,0,1,1], 
              [0,0,1,0]], dtype=float)

# in the A matrix, replace all 0 with np.inf
# NOTE caveat: A is not float matrix and not as space efficient?
A[A == 0] = np.inf

# B = ( shingels x hash functions)
B = np.array([[1,1], 
              [2,4], 
              [3,2], 
              [4,0], 
              [0,3]])


# matrix magic
C = np.einsum('sd,sh->hsd', A, B)

# find all min values
np.nanmin(C, axis=1)

array([[1., 3., 0., 1.],
       [0., 2., 0., 0.]])

In [126]:
# sandbox to test how it scales
s, h, d = 10000, 100, 10

A = np.random.randint(2, size = (s, d)).astype("float")
A[A == 0] = np.inf

B = np.random.randint(1000, size = (s, h)).astype("float")

print("Time einsum")
%timeit einsum_signature(A,B)

print("Time naive")
%timeit naive_signature(A,B)


Time einsum
13.3 ms ± 569 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time naive
3.41 s ± 51.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# LSH

With the signature matrix at hand, we are supposed to reduce the number of required comparisons by generating *candidate pairs*, which much likely are going to be similar. This is done by using yet another matrix, the matrix *M*. Approach according to lecture:

* Divide matrix *M* into *b* bands and *r* rows
* For each band, hash its portion of each column to a hash table with *k* buckets (make *k* as large as possible)
* Candidate column pair are those that hash to the same bucker for ≥ 1 band.
* tune *b* and *r* to catch most similar pairs, but few non-similar pairs. 

Recommender *b* and *r* values, but for signatare matrix of 100 hashes *b* = 20, *r* = 5.

> Statistiken gås igenom efter ca 1h 10m på föreläsningen. Förstår ej.  

In [ ]:
def find_candidate_pairs():
    return null